<a href="https://colab.research.google.com/github/hyeonjun/AlphaZero/blob/master/5_3_Monte_Carlo_Search_TicTacTo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 원시 몬테카를로 탐색이란?

알파베타법 같은 경우 탐색의 가지를 쳐내어 틱택토와 같은 국면이 적은 게임이라면 문제가 없으나, 장기나 체스와 같이 국면이 많은 게임에서는 막대한 시간이 소요되기 때문에 현실적이지 않다.

여기에서 수를 전개하는 부분을 더 줄여서 상태 가치를 계산하는 방법을 생각할 수 있다.

이를 실현하는 방법으로는 '수제작 평가 함수'와 원시 몬테카를로 탐색(Monte Carlo Search), 몬테카를로 트리 탐색(Monte Carlo Tree Search)을 꼽을 수 있다.


* 직접 구현한 평가 함수

'직접 구현한 평가 함수'란 장기에서 걸음 수가 4 이상이면 1점, 장기말이 움직일 수 있는 위치가 많은 1점과 같이 프로그래머가 스스로 계산 방법을 생각해 평가 함수를 만드는 방법이다. 강력한 평가 함수를 만들기 위해서는 우선 평가 함수를 작성하는 사람이 게임을 잘 알고 있어야 하며, 또한 그 지식을 알고리즘으로 만들어야 하기 때문에 난이도가 매우 높다.

* 원시 몬테카를로 탐색, 몬테카를로 트리 탐색

'원시 몬테카를로 탐색', '몬테카를로 트리 탐색'은 랜덤 시뮬레이션을 활용해 상태 가치를 계산하는 방법이다. 몬테카를로라는 말이 어렵게 들리지만, 이것은 랜덤이라는 의미를 말한다.

현재 국면에서 게임 종료 시까지 계속해서 랜덤 플레이를 수행한 뒤, 승률이 높은 수가 가치가 높은 것으로 판단한다. 몬테카를로 트리 탐색은 원시 몬테카를로 탐색을 개선한 알고리즘이다.

이번에는 원시 몬테카를로 탐색을 사용해볼 것이다.

In [ ]:
# 틱택토 구현
import random

# 게임 상태
class State:
    # 초기화
    def __init__(self, pieces=None, enemy_pieces=None):
        # 돌 배치
        self.pieces = pieces if pieces != None else [0] * 9
        self.enemy_pieces = enemy_pieces if enemy_pieces != None else [0] * 9

    # 돌의 수 취득
    def piece_count(self, pieces):
        count = 0
        for i in pieces:
            if i == 1:
                count +=  1
        return count

    # 패배 여부 확인
    def is_lose(self):
        # 돌 3개 연결 여부
        def is_comp(x, y, dx, dy):
            for k in range(3):
                if y < 0 or 2 < y or x < 0 or 2 < x or \
                    self.enemy_pieces[x+y*3] == 0:
                    return False
                x, y = x+dx, y+dy
            return True

        # 배패 여부 확인
        if is_comp(0, 0, 1, 1) or is_comp(0, 2, 1, -1):
            return True
        for i in range(3):
            if is_comp(0, i, 1, 0) or is_comp(i, 0, 0, 1):
                return True
        return False

    # 무승부 여부 확인
    def is_draw(self):
        return self.piece_count(self.pieces) + self.piece_count(self.enemy_pieces) == 9

    # 게임 종료 여부 확인
    def is_done(self):
        return self.is_lose() or self.is_draw()

    # 다음 상태 얻기
    def next(self, action):
        pieces = self.pieces.copy()
        pieces[action] = 1
        return State(self.enemy_pieces, pieces)

    # 합법적인 수의 리스트 얻기
    def legal_actions(self):
        actions = []
        for i in range(9):
            if self.pieces[i] == 0 and self.enemy_pieces[i] == 0:
                actions.append(i)
        return actions

    # 선 수 여부 확인
    def is_first_player(self):
        return self.piece_count(self.pieces) == self.piece_count(self.enemy_pieces)

    # 문자열 표시
    def __str__(self):
        ox = ('o', 'x') if self.is_first_player() else ('x', 'o')
        str = ''
        for i in range(9):
            if self.pieces[i] == 1:
                str += ox[0]
            elif self.enemy_pieces[i] == 1:
                str += ox[1]
            else:
                str += '-'
            if i % 3 == 2:
                str += '\n'
        return str

In [ ]:
# 랜덤으로 행동 선택( Mini-Max-Method)
def random_action(state):
    legal_actions = state.legal_actions()
    return legal_actions[random.randint(0, len(legal_actions)-1)]

In [ ]:
# 알파베타법을 활용한 상태 가치 계산
def alpha_beta(state, alpha, beta):
    # 패배 시, 상태 가치 -1
    if state.is_lose():
        return -1
    
    # 무승부 시, 상태 가치 0
    if state.is_draw():
        return  0

    # 합법적인 수의 상태 가치 계산
    for action in state.legal_actions():
        score = -alpha_beta(state.next(action), -beta, -alpha)
        if score > alpha:
            alpha = score

        # 현재 노드의 베스트 스코어가 새로운 노드보다 크면 탐색 종료
        if alpha >= beta:
            return alpha

    # 합법적인 수의 상태 가치의 최대값을 반환
    return alpha

# 알파베타법을 활용한 행동 선택
def alpha_beta_action(state):
    # 합법적인 수의 상태 가치 계산
    best_action = 0
    alpha = -float('inf')
    for action in state.legal_actions():
        score = -alpha_beta(state.next(action), -float('inf'), -alpha)
        if score > alpha:
            best_action = action
            alpha = score
            
    # 합법적인 수의 상태 가치값 중 최대값을 선택하는 행동 반환
    return best_action

# 플레이아웃

현재 국면에서 게임 종료까지 플레이하는 것을 플레이아웃(playout)이라고 부른다. 게임 종료까지 둘 수 있는 수를 랜덤으로 선택해서 상태 가치 '1: 승리', '-1: 패배', '0: 무승부'을 반환하는 함수를 만든다.

체스나 장기의 경우, 랜덤으로 수를 선택하게 되면 승부가 날 때까지 매우 많은 수가 필요할 것이라 생각되는데, 실제로는 그렇게 오래 걸리지 않는다고 한다.

예를 들면, 체스의 경우 임의의 국면에서의 둘 수 있는 수는 평균 35로, 평균 80수 정보에서 승부가 난다. 그러므로 모든 노드를 탐색하려면 35^80, 즉 10^120수 정도의 계산을 해야한다. 하지만 플레이아웃이라면 1회당 80수 정도의 계산만으로 완료할 수 있다.



In [ ]:
# 플레이아웃
def playout(state):
  # 패배 시, 상태 가치는 -1
  if state.is_lose():
    return -1
  
  # 무승부 시, 상태 가치는 0
  if state.is_draw():
    return 0

  # 다음 상태의 상태 평가
  return -playout(state.next(random_action(state)))

# 원시 몬테카를로 탐색을 활용한 행동 선택

원시 몬테카를로 탐색에서의 상태 가치 계산을 수행한다. 여기서는 둘 수 있는 수별로 10회 플레이아웃을 했을 때 상태 가치의 합계를 계산한다. 그리고 합계가 가장 큰 행동을 선택. 플레이아웃 횟수가 많을수록 정밀도가 높아지지만, 그만큼 시간도 오래 걸린다.


In [ ]:
# 원시 몬테카를로 탐색을 활용한 행동 선택
def mcs_action(state):
  # 합법적인 수 별로 10회 플레이아웃 시행 후, 상태 가치의 합계 계산
  legal_actions = state.legal_actions()
  values = [0] * len(legal_actions)
  for i, action in enumerate(legal_actions):
    for _ in range(10):
      values[i] += -playout(state.next(action))
    
  # 합법적인 수의 상태 가치 합계의 최대값을 가지는 행동 반환
  return legal_actions[argmax(values)]



argmax()는 집합 중 최댓값의 인덱스를 반환하는 함수. 예를 들면, 'argmax([2,5,3])'이라면 최댓값은 5이므로 그 값의 인덱스인 1을 반환한다. 이 값을 사용해 상태 가치의 합계가 가장 큰 행동을 선택한다.

In [ ]:
# 최대값의 인덱스를 반환
def argmax(collection, key=None):
  return collection.index(max(collection))

# 원시 몬테카를로 탐색과 랜덤 및 알파베타법의 대전

원시 몬테카를로 탐색과 랜덤 및 알파베타법을 대전시킨다. 게임을 100회 플레이하고 그 승률을 표시한다. 또한 틱택토에서는 선 수가 유리하므로 선/후 수를 교대한다.

In [ ]:
# 파라미터
EP_GAME_COUNT = 100 # 평가 1회당 게임 수

# 선 수 플레이어 포인트
def first_player_point(ended_state):
  # 1: 선 수 플레이어 승리, 0: 선 수 플레이어 패배, 0.5: 무승부
  if ended_state.is_lose():
    return 0 if ended_state.is_first_player() else 1
  return 0.5

# 게임 실행
def play(next_actions):
  # 상태 생성
  state = State()

  # 게임 종료 시까지 반복
  while True:
    # 게임 종료 시
    if state.is_done():
      break
    
    # 행동 얻기
    next_action = next_actions[0] if state.is_first_player() else next_actions[1]
    action = next_action(state)

    # 다음 상태 얻기
    state = state.next(action)

  # 선 수 플레이어 포인트 반환
  return first_player_point(state)

# 임의의 알고리즘 평가
def evaluate_algorithm_of(label, next_actions):
  # 여러 차레 대전 반복
  total_point = 0
  for i in range(EP_GAME_COUNT):
    # 1 게임 실행
    if i % 2 == 0:
      total_point += play(next_actions)
    else:
      total_point += 1-play(list(reversed(next_actions)))
    
    # 출력
    print('\rEvaluate {}/{}'.format(i+1, EP_GAME_COUNT), end='')
  print('')

  # 평균 포인트 계산
  average_point = total_point / EP_GAME_COUNT
  print(label.format(average_point))


# 랜덤과의 대전
next_actions = (mcs_action, random_action)
evaluate_algorithm_of('VS_Random {:.3f}', next_actions)

# 알파베타법과의 대전
next_actions = (mcs_action, alpha_beta_action)
evaluate_algorithm_of('VS_AlphaBeta {:.3f}', next_actions)

Evaluate 100/100
VS_Random 0.935
Evaluate 100/100
VS_AlphaBeta 0.290


위 결과를 보면 원시 몬테카를로 검색은 랜덤과의 대전에는 압승이지만, 알파베타법과의 대전에서는 패배하는 것을 볼 수 있다. 틱택토에서는 재귀적으로 게임 종료 시까지 조사하는 알파베타법이 가장 강하기 때문에 다른 알고리즘으로 이기기엔 상당히 어렵다.